In [ ]:
%run "0a_Workspace_setup.ipynb"
#%run "0b_Create_poi_files.ipynb"

from folium.features import DivIcon
#from NHM_helpers.NHM_hydrofabric import *


In [ ]:
hru_gdf = create_hru_gdf(
    model_dir,
    GIS_format,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
)

seg_gdf = create_segment_gdf(
    model_dir,
    GIS_format,
    param_filename,
)

nwis_gages_aoi = fetch_nwis_gage_info(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
)

poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    nwis_gages_aoi,
    gages_file,
)

default_gages_file = create_default_gages_file(
    model_dir,
    nwis_gages_aoi,
    poi_df,
)

gages_df = read_gages_file(
    model_dir,
    poi_df,
    nwis_gages_file,
    gages_file,
)

## Set default aspects of maps

### Set approximate latitude, longitude and zoom level for subbasin is calculated for starting point of folium.map plot window.

In [ ]:
pfile_lat = hru_gdf["hru_lat"].mean()
pfile_lon = hru_gdf["hru_lon"].mean()
zoom = 8

### Set up a custom tiles for base map backgrounds
This can be tricky with syntax but if you go to this link you will find resources that have options beyond the few defualt options in folium leaflet, 

http://leaflet-extras.github.io/leaflet-providers/preview/

These tiles will also work in the minimap, but can get glitchy if the same tile var is used in the minimap and the main map child object.

In [ ]:
USGStopo_layer = folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}",
    attr="USGS_topo",
    zoom_start=zoom,
    name="USGSTopo",
    show=False,
)

USGSHydroCached_layer = folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSHydroCached/MapServer/tile/{z}/{y}/{x}",
    attr="USGSHydroCached",
    zoom_start=zoom,
    name="USGSHydroCached",
)

### Set up inset map
This requires folium plugins. (from folium import plugins)

In [ ]:
minimap = plugins.MiniMap(
    tile_layer="OpenStreetMap",
    # attr = 'USGS_topo',
    position="topleft",
    # zoom_level_offset=- 4,
    height=200,
    width=200,
    collapsed_height=25,
    collapsed_width=25,
    zoom_level_fixed=5,
    toggle_display=True,
    # collapsed = True
)

### Set global mapping style functions

In [ ]:
transparent = lambda x: {
    "fillColor": "#00000000",
    "color": "#00000000",
    "weight": 4,
}

### Set HRU mapping styles

In [ ]:
# Style functions
style_function_hru_map = lambda x: {
    "opacity": 1,
    "fillColor": "#00000000",  #'goldenrod',
    "color": "tan",
    "weight": 1.5,
}
highlight_function_hru_map = lambda x: {
    "opacity": 0.5,
    "color": "gray",
    "fillColor": "gray",
    "weight": 3,
}

tooltip_hru = folium.GeoJsonTooltip(
    fields=["nhm_id", "hru_segment_nhm"],
    aliases=["hru", " flows to segment"],
    labels=True,
    # style=("background-color: #F0EFEF;border: 2px solid black;font-family: arial; font-size: 16px; padding: 10px;"),# Note that this tooltip style sets the style for all tool_tips.
)

popup_hru = folium.GeoJsonPopup(
    fields=["nhm_id", "hru_segment_nhm"],
    aliases=["hru", " flows to segment"],
    labels=True,
    localize=False,
    style=(
        "font-size: 16px;"
    ),  # Note that this tooltip style sets the style for all tool_tips.
    # background-color: #F0EFEF;border: 2px solid black;font-family: arial; padding: 10px; background-color: #F0EFEF;
)

hru_map = folium.GeoJson(
    hru_gdf,
    style_function=style_function_hru_map,
    highlight_function=highlight_function_hru_map,
    name="NHM HRUs",
    # tooltip=tooltip_hru,
    popup=popup_hru,
)


marker_radius = 5

### Set segment mapping styles

In [ ]:
####################################### Segments #########################################################
style_function_seg_map = lambda x: {"opacity": 1, "color": "#217de7", "weight": 2}
highlight_function_seg_map = lambda x: {"opacity": 1, "color": "white", "weight": 4}

tooltip_seg = folium.GeoJsonTooltip(
    fields=["nhm_seg", "tosegment_nhm"],
    aliases=["segment", "flows to segment"],
    labels=True,
)
popup_seg = folium.GeoJsonPopup(
    fields=["nhm_seg", "tosegment_nhm"],
    aliases=["segment", "flows to segment"],
    labels=True,
    localize=False,
)

seg_map_show = folium.GeoJson(
    seg_gdf,
    style_function=style_function_seg_map,
    highlight_function=highlight_function_seg_map,  # lambda feature: {"fillcolor": "white", "color": "white"},
    name="NHM Segments",
)

# This makes a transparent layer that will always be on the top and so the tooltip is associated with this layer, and will also always be on top.
seg_map_hide = folium.GeoJson(
    seg_gdf,
    style_function=transparent,
    highlight_function=highlight_function_seg_map,  # lambda feature: {"fillcolor": "white", "color": "white"},
    name="NHM Segments",
    # tooltip=tooltip_seg,
    popup=popup_seg,
)

### Set poi (gages) mapping style

In [ ]:
# add POI marker cluster child items for the map
poi_marker_cluster = MarkerCluster(
    name="Model poi",
    overlay=True,
    control=True,
    icon_create_function=None,
    disableClusteringAtZoom=7,
)
poi_marker_cluster_label = MarkerCluster(
    name="Model poi label",
    overlay=True,
    control=True,
    show=False,  # False will not draw the child upon opening the map, but have it to draw in the Layer control.
    icon_create_function=None,
    disableClusteringAtZoom=7,
)
##add POI markers and labels using row df.interowss loop
for idx, row in poi_df.iterrows():
    text = f'{row["poi_id"]}'
    label_lat = row["latitude"]  # -0.01
    label_lon = row["longitude"]

    marker_label = folium.map.Marker(
        [label_lat, label_lon],
        icon=DivIcon(
            icon_size=(10, 10),  # (150,36),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12pt; font-weight: bold">%s</div>' % text,
        ),
    ).add_to(poi_marker_cluster_label)

    marker = folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        name=row["poi_id"],
        popup=folium.Popup(
            f'<font size="3px">{row["poi_id"]} ({row["poi_agency"]})<br>{row["poi_name"]}<br> on <b>segment </b>{row["nhm_seg"]}</font>',
            max_width=280,
            max_height=2000,
        ),
        radius=5,
        weight=2,
        color="black",
        fill=True,
        fill_color="Yellow",
        fill_opacity=1.0,
    ).add_to(poi_marker_cluster)

### Set non-poi (gages) mapping style

In [ ]:
# add non-poi gages marker cluster child items for the map
non_poi_marker_cluster = MarkerCluster(
    name="Non-poi",
    overlay=True,
    control=True,
    icon_create_function=None,
    disableClusteringAtZoom=7,
)
non_poi_marker_cluster_label = MarkerCluster(
    name="Non-poi label",
    overlay=True,
    control=True,
    show=False,  # False will not draw the child upon opening the map, but have it to draw in the Layer control.
    icon_create_function=None,
    disableClusteringAtZoom=7,
)

##add Non-poi gage markers and labels using row df.interowss loop
gages_list = gages_df.index.to_list()
additional_gages = list(set(gages_list) - set(poi_df.poi_id))


for idx, row in nwis_gages_aoi.iterrows():
    if row["poi_id"] in additional_gages:

        text = f'{row["poi_id"]}'
        label_lat = row["latitude"]  # -0.01
        label_lon = row["longitude"]

        marker_label = folium.map.Marker(
            [label_lat, label_lon],
            icon=DivIcon(
                icon_size=(10, 10),  # (150,36),
                icon_anchor=(0, 0),
                html='<div style="font-size: 12pt; font-weight: bold">%s</div>' % text,
            ),
        ).add_to(non_poi_marker_cluster_label)

        marker = folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            name=row["poi_id"],
            popup=folium.Popup(
                f'<font size="3px">{row["poi_id"]} ({row["poi_agency"]})<br>{row["poi_name"]}<br></font>',
                max_width=280,
                max_height=2000,
            ),
            radius=5,
            weight=2,
            color="black",
            fill=True,
            fill_color="Red",
            fill_opacity=1.0,
        ).add_to(non_poi_marker_cluster)
    else:
        pass

In [ ]:
# crs = hru_gdf.crs

## Create interactive HRU map

In [ ]:
from IPython.display import display

m2 = folium.Map(
    location=[pfile_lat, pfile_lon],
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    width="100%",
    height="100%",
    control_scale=True,
)

USGStopo_layer.add_to(m2)

# add HRU maps
# hru_gdf[['nhm_id','hru_segment_nhm']] = hru_gdf[['nhm_id','hru_segment_nhm']].astype(object)

hru_map.add_to(m2)

seg_map_show.add_to(m2)

# This makes a transparent layer that will always be on the top and so the tooltip is associated with this layer, and will also always be on top (last mapped).
seg_map_hide.add_to(m2)


# Add tool tips to map
# hru_map.add_child(tooltip_hru)
# seg_map_hide.add_child(tooltip_seg)

poi_marker_cluster.add_to(m2)
poi_marker_cluster_label.add_to(m2)


# Add tool tips to map
# hru_map.add_child(tooltip_hru)
# seg_map_hide.add_child(tooltip_seg)

non_poi_marker_cluster.add_to(m2)
non_poi_marker_cluster_label.add_to(m2)

m2.add_child(minimap)
plugins.Fullscreen(position="bottomleft").add_to(m2)
folium.LayerControl(collapsed=True, position="bottomright").add_to(m2)

# display(m2)

con.print(
    f"[underline]The {subbasin} Model Hydrofabric[/underline]\n",
    f"\n{pdb.dimensions.get('nhru').meta['description']} (nhru), {pdb.dimensions.get('nhru').meta['size']}",
    f"\n{pdb.dimensions.get('nsegment').meta['description']} (nsegment), {pdb.dimensions.get('nsegment').meta['size']}",
    f"\n{pdb.dimensions.get('npoigages').meta['description']} (npoigages), {pdb.dimensions.get('npoigages').meta['size']}",
    f"\nNumber of Non-poi gages, {len(additional_gages)}",
)

display(m2)